<a href="https://colab.research.google.com/github/meghutch/Breast-Cancer-Classification-Clinical-Genomic/blob/master/Gene_Expression_Analysis_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Clinical Outcomes of Breast Cancer Patients - Gene Expression Data**

## **Principal Component Analysis**

**Author:** Meg Hutch

**Date:** November 19, 2019

**Objective:** Apply Principal Component Analysis to Determine Which Genes to Include into our Neural Network 

****Update: I previously did some testing and these methods don't seem to be causing significant differences. I will probably just need to later validate whether the PC change using the different methods, but for now, I think I can go ahead, clean up the code****

**References**

* https://www.youtube.com/watch?v=Lsue2gEM9D0&list=PLblh5JKOoLUIcdlgu78MnlATeyx4cEVeR&index=54&t=0s

* https://www.youtube.com/watch?v=FgakZw6K1QQ

* https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60 

* https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python 

* https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c

In [0]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

In [11]:
# Connect Colab to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Import Data
gene_data = pd.read_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Data/merged_expression.txt', sep=',')

In [0]:
#gene_data.head()
#genes.info()

# **Data Pre-Processing**

**Check if NAs**

In [14]:
gene_data.isna().any()

Unnamed: 0    False
EVENT          True
OS_MONTHS     False
FIVE_YEAR      True
RERE          False
              ...  
CC2D1A        False
CB986545      False
IGSF9         False
DA110839      False
FAM71A         True
Length: 24372, dtype: bool

**Remove cases where there are missing values**

In [0]:
gene_data = gene_data.dropna()

**Check number of patients after removing Nas**

In [16]:
gene_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1747 entries, 0 to 1903
Columns: 24372 entries, Unnamed: 0 to FAM71A
dtypes: float64(24371), object(1)
memory usage: 324.9+ MB


**Create a dataset just containing gene expression**

In [0]:
# remove event, os_motnhs, and Five_Year from the dataset
genes = gene_data.drop(columns=["Unnamed: 0", "EVENT", "OS_MONTHS", "FIVE_YEAR"])

**Create a dataset containing the target variables**

In [18]:
# Subset only the Event - whether they lived (0) or died (1) from Breast Cancer
labels = gene_data.EVENT

# Create a list of row names
patients = list(genes.index)

# Convert labels into a dataframe and indicate patients as the index
labels = pd.DataFrame(labels, index = patients)

print(labels)

      EVENT
0       0.0
1       0.0
2       1.0
3       0.0
4       1.0
...     ...
1899    0.0
1900    1.0
1901    1.0
1902    0.0
1903    0.0

[1747 rows x 1 columns]


**Save Gene Dataset**

Save the genetic data sets that did not have missing values

In [0]:
gene_data.to_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Processed_Data/all_gene_data.txt')

# **Principal Component Analysis**

**What is a Pricipal Component? (From DataCamp)**

Principal componenets have both direction and magnitude. The direction represents across which principal axes the data is mostly spread out or has the most variance and the magnitude signifies the amount of varaince that Principal Component captures of the data when projected onto that axis. 

The principal components are a straight line, and the first principal component holds the most variance in the data. Each subsequent prinicpal component is orthogonal to the last and has a lesser variance .

Correlated features contribute to the same principal component, thereby reducing the original data features into uncorrelated prinicpal components; each representing a different set of correlated features with differents of variation.

Each principal component represents a percentage of total variation captured from the data

****Note:** I troubleshooted some discrepancies among the references above in regards to fit and fit transform. Some articles would initially use fit_transform to scale and then again used pca.fit_transform when identifying prinicipal components, other references, instead of fit_transform just used pca.transform for this step. I tested both methods and got the same results. Actually looks like maybe using fit_transform is probably just combining these steps?****** 


**1) Standardize the Data**

Must scale features in your data before applying PCA. **StandardScaler** helps standardize features onto unit scale (mean = 0 and standard deviation = 1). Thus, each value in the dataset will have the sample mean value subtracted and then divided by the standard deviation of the whole dataset. 


In [0]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

# Stanardize/Scale the data
x = StandardScaler().fit_transform(genes)

**Let's check whether the normalized data has a mean of zero and a standard deviation of 1**

In [0]:
np.mean(x), np.std(x)

**Convert the normalized features into tabular format**

In [0]:
# Create list of column names
features = list(genes.columns.values) 

# Create data frame of newly normalized data - use patients IDs as the index 
x = pd.DataFrame(x, columns = features, index = patients)

**2) Determine Prinicpal Components**

Reference: https://stackoverflow.com/questions/42167907/understanding-scikitlearn-pca-transform-function-in-python

**pca.fit** allows PCA function to compute vectors that you can project your data onto in order to reduce the dimension of your data. 

**pca.transform** actually performs the projection. It projects each row of data into the vector space that was learned when fit was called.

from sklearn: **fit_transform**: Fit the model with X and apply the dimensionality reduction on X

In [0]:
# Define pca function
pca = PCA()

# Fit to the scaled/standardized data - then use transform to prokect into the new vector space learned by fit
principalComponents = pca.fit_transform(x)

# Generate a list of column names with the number for each prinicpal component 
col_names = [f'pc{i}' for i in range(1, 1748)] # there are 1747 samples - so we want to have range of 1 less than 1748 column names 

# Add column names to the principal component dataset 
principalDf = pd.DataFrame(principalComponents, columns = col_names, index = patients)

In [0]:
principalDf 

**3) Determine # of Components and Variance**

In [0]:
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Gene Expression Explained Variance')
plt.show()

The plot tells us that with ~1250 components we can capture 90% of the data. 

**Save All Principal Components**

In [0]:
principalDf.to_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Processed_Data/gene_pca_components_All_1747.txt')

**Alternative method - Pre-selecting % of variance**

**Fit PCA to the data**

In [0]:
pca = PCA(0.9)
x2 = pca.fit_transform(x)

In [0]:
x2 = pd.DataFrame(data = x2)
x2 #this is the 1213 principal components

**Determine the exact number of n_components needed to capture 0.9 variance**

In [0]:
pca.n_components_

This function indicates that 1213 is the number of principal componenets needed to capture 90% of the variation which is what I had estimated from the above plot.

**Scree Plot**

View which principal components contribute most to the variance 

In [0]:
# remove PC from 
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
per_var = per_var[:10] #top 10 PC - this number is chosen just so that we can more easily view the plot
labels = col_names[:10]

plt.bar(x=range(1, len(per_var)+1), height = per_var, tick_label = labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Prinicpal Component')
plt.title('Scree Plot')
plt.show()

**Draw PCA Plot**

With the two primary principal components

References: 

https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python

https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [0]:
# Can use the function below with .sample to select a subset of PC components, making it more readible
principalDf_samp = principalDf.sample(n = 500) # will need to change the dataframe principalDf below to prinicpalDf_samp if I use this function

#plt.scatter(principalDf.pc1, principalDf.pc2)
plt.title('Principal Components')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))

# Replace Labels
gene_data['EVENT'].replace(0, 'Lived',inplace=True)
gene_data['EVENT'].replace(1, 'Died',inplace=True)

# Create labels for the targets to be used to color the graphs
targets = ['Lived', 'Died']
colors = ['b', 'r']

for target, color in zip(targets, colors):
    indicesToKeep = gene_data['EVENT'] == target
    plt.scatter(principalDf.loc[indicesToKeep, 'pc1'],
                principalDf.loc[indicesToKeep, 'pc2'], c = color)

# The labeled numbers are the indiviudal patients samples
for sample in principalDf.index:
  plt.annotate(sample, (principalDf.pc1.loc[sample], principalDf.pc2.loc[sample])) # my impression is that sample just indicates each row

# Subset
#sample_50 = principalDf.sample(n = 50)

# This plot is interesting -- it looks like it's creating random samples, but still impossing it on the main graph because we are indeicating pc1 and pc2 still which all samples would be under
#for sample in sample_50.index:
#  plt.annotate(sample, (sample_50.pc1.loc[sample], sample_50.pc2.loc[sample]))
#print(sample_50)

#print(principalDf.sample) -- I believe sample is returning just all of the row labels -- hard to read though which is why I was trying to subset
#print(principalDf.sample(n = 50))

**Determine Relevant Genes**

Get the name of the top 10 genes that contribute most to pc1. The following code looks at the components of each pca - which is also called the loading scores. Sorting by the absolute value, allows us to identify which genes, in either the negative or positive direction, had the most infleunce on each prinicipal component.



References: 

https://stackoverflow.com/questions/47370795/pca-on-sklearn-how-to-interpret-pca-components

https://www.youtube.com/watch?v=FgakZw6K1QQ

In [0]:
## first, get the loading scores
loading_scores = pd.Series(pca.components_[0], index=features)

## now sort the loading scores based on their magnitude
sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)
 
# get the names of the top 10 genes
top_10_genes = sorted_loading_scores[0:10].index.values
 
## print the gene names and their scores (and +/- sign)
print(loading_scores[top_10_genes])

**Loading Scores Test**

We can also validate the above loading scores function by looking at the values obtained when just using pca.fit. Firsrt, we perform just the fit function on the original x which is the scaled/standardized data. Then we can view the compoenent/loading scores which allow us to examine which genes had the most influence for each component. It is the absolute value 

In [0]:
# Test
principalComponents = pca.fit(x)
df_test = pd.DataFrame(pca.components_, columns=list(x.columns))  
print(df_test.SEPT15)
print(df_test.GLRX3)

In [0]:
pca.components_.shape # when we look at the shape, there are 1213 rows (components) and 24368 columns for the gene features 

**Select the Top 1213 Prinicipal Components that Explain 90 % of the Variance**

In [0]:
principalDf = principalDf.iloc[:,0:1213]

In [0]:
principalDf

**Save the selected PCA Components**

In [0]:
principalDf.to_csv('/content/drive/My Drive/Projects/Breast_Cancer_Classification/Processed_Data/gene_pca_components_90.txt')